In [ ]:
!pip install kfp --upgrade

In [1]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics)

from kfp.v2 import compiler

from preproc_split_dataset import preproc_split_dataset
from train import train_model
from evaluate_model import evaluate_model
#from helper_components import build_image
#from helper_components import custom_deploy

REGION = 'us-central1'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [ ]:
!gsutil mb -p $PROJECT_ID  gs://kubeflow-lab-v2
!gsutil ls

In [2]:
root_path = "gs://kubeflow-lab-v2"

@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=root_path ,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="pipeline-test-1")


def pipeline(alpha_val:float=0.001,max_iter_val:int=100,root_path:str=root_path):
    dataset_op = preproc_split_dataset(root_path=root_path)
    train_op = train_model(dataset_train=dataset_op.outputs["dataset_train"],
                           dataset_validation=dataset_op.outputs["dataset_validation"],
                           alpha=alpha_val,
                           max_iter=max_iter_val,
                           root_path=root_path
                          )
    eval_op = evaluate_model(
        test_set=dataset_op.outputs["dataset_test"],
        model_artifact=train_op.outputs["model_artifact"],
        root_path=root_path
    )

In [3]:
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='sklearn_pipe.json')